## Miniflow Architecture



# Implementing Forward propagation
x+y and (x+y)+y

In [31]:
class Node(object):
    def __init__ (self,inbound_nodes = []):
        # Node(s) from which this node recieves values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this node passes values
        self.outbound_nodes = []
        # Setting the current node as outbound node of it's inbound node
        self.value = None
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        print("\n I am inside Node class: ",self.__dict__)

    # These will be implemented in a subclass.
    # Each node will need to be able to pass values forward and perform backpropagation.
    # Overwrite the value if one is passed in.
    def forward(self, value = None):
        """
        Forward propagation.

        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        """
        raise NotImplemented

# NOTE: Input node is the only node where the value.
# may be passed as an argument to forward().
# The Input subclass just holds a value, such as a data feature or a model parameter (weight/bias).
class Input(Node):
    def __init__(self):
        # An Input Node has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Node.__init__(self,[])
        print("I am inside Input class: ",self)
        
    # NOTE: Input node is the only node that may
    # receive its value as an argument to forward().
    #
    # All other node implementations should calculate their
    # values from the value of previous nodes, using
    # self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        print("\n I am inside Input class forward method: ",self.value)
        if value is not None:
            self.value = value
            


        
# Add, which is another subclass of Node, actually can perform a calculation (addition).

class Add(Node):
    def __init__ (self,x,y):
        Node.__init__(self,[x,y])
        print("I am inside ADD class: " ,self)
        print("Setting outbound for input y:", y.__dict__)
        print("\n Setting outbound for input x:", x.__dict__)


    # Each node will need to be able to pass values forward and perform backpropagation
    def forward(self):
        x_value = self.inbound_nodes[0].value
        y_value = self.inbound_nodes[1].value
        self.value = x_value + y_value
        print("\n I am inside ADD class forward method: " ,self.value)


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L



# Forward pass runs the network and outputs a value.
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """
    print("I am inside forward pass method and printing output_node:",output_node)
    print("I am inside forward pass method and printing sorted_nodes :",sorted_nodes)
    for n in sorted_nodes:
        print("\n I am inside method forward pass loop and printing n in sorted_nodes:" ,n.__dict__)
        n.forward()
        #print("I am inside method forward pass loop and printing forward_pass forward(): " ,n.forward())
    
    return output_node.value


           

In [32]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y  # in case of this we can take one more Add(f,y)
"""

#from miniflow import *

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort sets the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))



 I am inside Node class:  {'inbound_nodes': [], 'outbound_nodes': [], 'value': None}
I am inside Input class:  <__main__.Input object at 0x0000012C0C9E5850>

 I am inside Node class:  {'inbound_nodes': [], 'outbound_nodes': [], 'value': None}
I am inside Input class:  <__main__.Input object at 0x0000012C0C9DEE20>

 I am inside Node class:  {'inbound_nodes': [<__main__.Input object at 0x0000012C0C9E5850>, <__main__.Input object at 0x0000012C0C9DEE20>], 'outbound_nodes': [], 'value': None}
I am inside ADD class:  <__main__.Add object at 0x0000012C0C9E5BE0>
Setting outbound for input y: {'inbound_nodes': [], 'outbound_nodes': [<__main__.Add object at 0x0000012C0C9E5BE0>], 'value': None}

 Setting outbound for input x: {'inbound_nodes': [], 'outbound_nodes': [<__main__.Add object at 0x0000012C0C9E5BE0>], 'value': None}
I am inside forward pass method and printing output_node: <__main__.Add object at 0x0000012C0C9E5BE0>
I am inside forward pass method and printing sorted_nodes : [<__main__

Unlike the other subclasses of Node, the Input subclass does not actually calculate anything. The Input subclass just holds a value, such as a data feature or a model parameter (weight/bias).



# Implementing (x+y)+y using above

In [33]:
class Node(object):
    def __init__(self,inbound_nodes = []):
        # Nodes from which this node recieves values
        self.inbound_nodes = inbound_nodes
        # Nodes to which this node passes values
        self.outbound_nodes = []
        # Setting this node as outbound node for all inbound nodes
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        self.value = None
        
    def forward():
        # Forward propagation compute output of the node based on 
        # inputs and store result in self.value
        
        raise notImplemented
        
# Input class has no inbound nodes.
class Input(Node):
    # Input nodes don't have inbound nodes so no need to pass anything in instatiator
    def __init__(self):
        Node.__init__(self)
        
    # Since there in no inbound in input node 
    # so we pass a value to forward propagation and 
    # assign it to the input node
    def forward(self,value = None):
        if value is not None:
            self.value = value

# The Add takes two inbound nodes and adds there values
class Add(Node):
    def __init__(self, x, y):
        Node.__init__(self,[x,y]) # calls Node's constructor
        
    def forward(self):
        """
        Set the value of this node (`self.value`) to the sum of its inbound_nodes.
        Remember to grab the value of each inbound_node to sum!

        Your code here!
        """
        #x_value = self.inbound_nodes[0].value
        #y_value = self.inbound_nodes[1].value
        #self.value = x_value + y_value
        self.value = 0
        for n in self.inbound_nodes:
            self.value += n.value
        
    
    

In [34]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: The output node of the graph (no outgoing edges).
        `sorted_nodes`: a topologically sorted list of nodes.

    Returns the output node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value


In [35]:
"""
This script builds and runs a graph with miniflow.


Implementing (x + y) + y
"""

# from miniflow import *

x, y = Input(), Input()

f = Add(x, y)

out = Add(f,y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(out, sorted_nodes)

# NOTE: because topological_sort sets the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[y], output))

10 + 5 + 5 = 20 (according to miniflow)
